In [45]:
import pandas as pd
import numpy as np

In [46]:
data = pd.read_csv('data/pitcher_subset.csv')

In [47]:
data['GameDate'] = pd.to_datetime(data['GameDate'])

Sort values chronologically and by game

In [48]:
data = data.sort_values(['GameDate', 'gameid', 'ab', 'pitchnum']).reset_index(drop=True)

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148535 entries, 0 to 148534
Data columns (total 37 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   gameid            148535 non-null  object        
 1   GameDate          148535 non-null  datetime64[ns]
 2   ab                148535 non-null  int64         
 3   pitchnum          148535 non-null  int64         
 4   inning            148535 non-null  int64         
 5   teambat           148535 non-null  int64         
 6   balls             148535 non-null  int64         
 7   strikes           148535 non-null  int64         
 8   outs              148535 non-null  int64         
 9   pitcher           148535 non-null  object        
 10  pitcherthrows     148535 non-null  object        
 11  batter            148535 non-null  object        
 12  batterside        148535 non-null  object        
 13  pitchresult       148535 non-null  object        
 14  pitc

Remove unnecessary columns (can easily be added back if needed)

In [50]:
data = data.drop(['spinrate', 'relspeed', 'horzbreak', 'inducedvertbreak', 'platelocside', 'platelocheight', 'season'], axis= 1 )

Check for NA values

In [51]:
data.isna().sum()

gameid                0
GameDate              0
ab                    0
pitchnum              0
inning                0
teambat               0
balls                 0
strikes               0
outs                  0
pitcher               0
pitcherthrows         0
batter                0
batterside            0
pitchresult           0
pitchresult_desc      0
eventtype             0
pitchname             0
pitchname_desc        0
hometeam_id           0
Home                  0
awayteam_id           0
Visitor               0
venue_id              0
venue_name            0
pitch_type            0
visscore            178
homscore            178
r1bool                0
r2bool                0
r3bool                0
dtype: int64

Only na values in visscore and homscore (and they occur at same time)

In [52]:
gameid_counts = data['gameid'].value_counts().reset_index()
na_gameid_counts = data[data['visscore'].isna()]['gameid'].value_counts().reset_index()
compare = pd.merge(na_gameid_counts, gameid_counts, on='gameid', how='left')
compare['values-not-missing'] = compare['count_y'] - compare['count_x']

In [53]:
compare.sort_values('values-not-missing').head(10)

,gameid,count_x,count_y,values-not-missing
0,2021/05/23/oakmlb-anamlb-1,13,13,0
2,2021/06/26/arimlb-sdnmlb-1,7,13,6
13,2021/06/26/oakmlb-sfnmlb-1,4,12,8
25,2021/06/04/wasmlb-phimlb-1,3,13,10
17,2023/09/09/sdnmlb-houmlb-1,4,14,10
23,2021/07/06/detmlb-texmlb-1,3,15,12
16,2021/07/06/bosmlb-anamlb-1,4,18,14
10,2022/06/26/houmlb-nyamlb-1,4,18,14
29,2023/06/21/oakmlb-clemlb-1,2,17,15
58,2021/04/07/milmlb-chnmlb-1,1,17,16


Only one game missing all values entirely, for the rest we can make the assumption that the score has not changed from the previous at bat

In [54]:
data.loc[(data['gameid'] == '2021/05/23/oakmlb-anamlb-1')][['pitcher', 'ab', 'pitchnum','inning', 'eventtype']]

,pitcher,ab,pitchnum,inning,eventtype
15487,"Guduan, Reymin",61,1,7,ball
15488,"Guduan, Reymin",61,2,7,ball
15489,"Guduan, Reymin",61,3,7,called_strike
15490,"Guduan, Reymin",61,4,7,sac_fly
15491,"Guduan, Reymin",62,1,7,called_strike
15492,"Guduan, Reymin",62,2,7,ball
15493,"Guduan, Reymin",62,3,7,single
15494,"Guduan, Reymin",63,1,7,ball
15495,"Guduan, Reymin",63,2,7,foul
15496,"Guduan, Reymin",63,3,7,foul


Doing further research online, we can find that the score was 4-5 for all these pitches, so fill these values

In [55]:
data.loc[(data['gameid'] == '2021/05/23/oakmlb-anamlb-1'), 'homscore'] = 5
data.loc[(data['gameid'] == '2021/05/23/oakmlb-anamlb-1'), 'visscore'] = 4

front fill the remaining values

In [56]:
data[['visscore', 'homscore']] = data[['visscore', 'homscore']].ffill()

In [57]:
data.isna().sum()

gameid              0
GameDate            0
ab                  0
pitchnum            0
inning              0
teambat             0
balls               0
strikes             0
outs                0
pitcher             0
pitcherthrows       0
batter              0
batterside          0
pitchresult         0
pitchresult_desc    0
eventtype           0
pitchname           0
pitchname_desc      0
hometeam_id         0
Home                0
awayteam_id         0
Visitor             0
venue_id            0
venue_name          0
pitch_type          0
visscore            0
homscore            0
r1bool              0
r2bool              0
r3bool              0
dtype: int64

No more NA values!

# Feature Building

In [58]:
features = data.copy()

Response (Pitch Type)

In [59]:
features['pitch_type'] = features['pitch_type'].map({'Breaking Ball': 0, 'Fastball':1}) #Encode as 1s and 0s for logistic regression response

Previous pitch / pattern recognition

In [60]:
features['prev_pitch'] = features['pitch_type'].shift(-1)

features.loc[(features['pitchnum'] == 1), 'last_pitch_na'] = 1 #don't want to previous pitch from one at bat to another
features['last_pitch_na'] = features['last_pitch_na'].fillna(0)

features.loc[(features['last_pitch_na'] == 0) & (features['prev_pitch'] == 1), 'last_pitch_fastball'] = 1
features['last_pitch_fastball'] = features['last_pitch_fastball'].fillna(0) #don't need one for breaking ball because it is reference level

features.loc[(features['last_pitch_na'] == 0) & ((features['eventtype'] == "called_strike") | (features['eventtype'] == "swinging_strike") | (features['eventtype'] == "foul")), 'last_pitch_strike'] = 1
features['last_pitch_strike'] = features['last_pitch_strike'].fillna(0) #don't need one for ball because it is reference level

features = features.drop('prev_pitch', axis=1)

Leverage (and variables that go into leverage)

In [61]:
features['score_diff'] = np.where(features['teambat'] == 0, features['homscore'] - features['visscore'], features['visscore'] - features['homscore'])

features['in_scoring_position'] = features[['r2bool', 'r3bool']].max(axis=1) #potential predictor seperate from leverage
features = features.rename({'r1bool':'on_first_base'}, axis=1)

features['inning_weight'] = (features['inning']-1) // 3 + 1
features.loc[features['inning_weight'] > 3, 'inning_weight'] = 3.0

features['r2bool'] = features['r2bool'].astype('bool')
features['r3bool'] = features['r3bool'].astype('bool')
features['on_first_base'] = features['on_first_base'].astype('bool')

features.loc[(features['on_first_base'] & ~features['r2bool'] & ~features['r3bool']), 'base_factor'] = 1.5
features.loc[(features['r2bool'] ^ features['r3bool'] & ~features['on_first_base']), 'base_factor'] = 2.0
features.loc[(features['r2bool'] & features['on_first_base'] & ~features['r3bool']) | (features['r3bool'] & features['on_first_base'] & ~features['r2bool']), 'base_factor'] = 2.25
features.loc[(features['r2bool'] & ~features['on_first_base'] & features['r3bool']), 'base_factor'] = 2.5
features.loc[(features['r2bool'] & features['on_first_base'] & features['r3bool']), 'base_factor'] = 2.75
features['base_factor'] = features['base_factor'].fillna(1.0)

features['leverage'] = features['inning_weight'] * (1 / (np.abs(features['score_diff']) + 1)) * features['base_factor']

Platoon Splits

In [62]:
#platoon_r_l is reference level

features.loc[(features['batterside'] == 'L') & (features['pitcherthrows'] == 'L'), 'platoon_l_l'] = 1.0
features['platoon_l_l'] = features['platoon_l_l'].fillna(0.0)
features.loc[(features['batterside'] == 'R') & (features['pitcherthrows'] == 'R'), 'platoon_r_r'] = 1.0
features['platoon_r_r'] = features['platoon_r_r'].fillna(0.0)
features.loc[(data['batterside'] == 'L') & (features['pitcherthrows'] == 'R'), 'platoon_l_r'] = 1.0
features['platoon_l_r'] = features['platoon_l_r'].fillna(0.0)

Batting Counts

In [ ]:
#binary variable for each specific count
features['balls_strikes'] = features['balls'].astype(int).astype(str) + '_' + features['strikes'].astype(int).astype(str)
one_hot_df = pd.get_dummies(features['balls_strikes'], prefix='Count', dtype=int)
features = pd.concat([features, one_hot_df], axis=1)

#used for grouping historical rates
features['batter_count'] = features[['Count_2_0', 'Count_3_0', 'Count_3_1']].max(axis=1) 
features['batter_count'] = features['batter_count'].fillna(0.0)
features['pitcher_count'] = features[['Count_0_2', 'Count_2_2', 'Count_1_2']].max(axis=1)
features['pitcher_count'] = features['pitcher_count'].fillna(0.0)

That's a lot of features!

In [64]:
features.columns

Index(['gameid', 'GameDate', 'ab', 'pitchnum', 'inning', 'teambat', 'balls',
       'strikes', 'outs', 'pitcher', 'pitcherthrows', 'batter', 'batterside',
       'pitchresult', 'pitchresult_desc', 'eventtype', 'pitchname',
       'pitchname_desc', 'hometeam_id', 'Home', 'awayteam_id', 'Visitor',
       'venue_id', 'venue_name', 'pitch_type', 'visscore', 'homscore',
       'on_first_base', 'r2bool', 'r3bool', 'last_pitch_na',
       'last_pitch_fastball', 'last_pitch_strike', 'score_diff',
       'in_scoring_position', 'inning_weight', 'base_factor', 'leverage',
       'platoon_l_l', 'platoon_r_r', 'platoon_l_r', 'balls_strikes',
       'Count_0_0', 'Count_0_1', 'Count_0_2', 'Count_1_0', 'Count_1_1',
       'Count_1_2', 'Count_2_0', 'Count_2_1', 'Count_2_2', 'Count_3_0',
       'Count_3_1', 'Count_3_2', 'batter_count', 'pitcher_count'],
      dtype='object')

In [65]:
features.to_csv('data/model_data.csv', index=False)